In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

root_dir = Path().absolute()
# Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
if root_dir.parts[-1:] == ('ccfraud',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
root_dir = str(root_dir) 

print(f"Root dir: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Root dir: /home/jdowling/Projects/mlfs-book
HopsworksSettings initialized!


In [2]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()

2025-12-26 08:01:32,458 INFO: Initializing external client
2025-12-26 08:01:32,460 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-26 08:01:33,790 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/120


In [3]:
merchant_fg = fs.get_feature_group("merchant_details", version=1)
account_fg = fs.get_feature_group("account_details", version=1)
bank_fg = fs.get_feature_group("bank_details", version=1)
card_fg = fs.get_feature_group("card_details", version=1)
cc_trans_aggs_fg = fs.get_feature_group("cc_trans_aggs_fg", version=1)
cc_trans_fg = fs.get_feature_group("cc_trans_fg", version=1)

In [15]:
# 'account_id', 'bank_id', 
#    .join(account_fg.select_all(), on="account_id")\
#cc_trans_aggs_fg.select_except(['prev_ip_transaction', 'prev_ts_transaction', 'event_time'])\
#     .join(bank_fg.select(['credit_rating', 'days_since_bank_cr_changed', 'country']), on="bank_id")
subtree1 = cc_trans_aggs_fg.select_all()\
    .join(bank_fg.select_all(), on="bank_id")

In [16]:
subtree1.show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.90s) 


,cc_num,account_id,bank_id,num_trans_last_10_mins,sum_trans_last_10_mins,num_trans_last_hour,sum_trans_last_hour,num_trans_last_day,sum_trans_last_day,num_trans_last_week,sum_trans_last_week,prev_card_present,prev_ip_transaction,prev_ts_transaction,event_time,bank_details_bank_id,bank_details_country,bank_details_credit_rating,bank_details_last_modified,bank_details_days_since_bank_cr_changed
0,1352-3884-5289-2657,None,None,1,83.81,1,83.81,1,83.81,1,83.81,False,50.69.81.2,NaT,2025-09-05 09:05:43,None,None,NaN,NaT,NaN
1,1352-3884-5289-2657,None,None,1,28.23,1,28.23,2,112.04,2,112.04,None,68.204.24.201,2025-09-05 09:05:43,2025-09-05 11:47:50,None,None,NaN,NaT,NaN
2,1352-3884-5289-2657,None,None,1,70.98,2,99.21,3,183.02,3,183.02,False,12.111.40.215,2025-09-05 11:47:50,2025-09-05 12:19:55,None,None,NaN,NaT,NaN
3,1352-3884-5289-2657,None,None,1,5.41,1,5.41,4,188.43,4,188.43,False,67.177.3.84,2025-09-05 12:19:55,2025-09-06 03:29:46,None,None,NaN,NaT,NaN
4,1352-3884-5289-2657,None,None,1,70.75,1,70.75,2,76.16,5,259.18,True,186.36.36.89,2025-09-06 03:29:46,2025-09-06 17:00:41,None,None,NaN,NaT,NaN
5,1352-3884-5289-2657,None,None,1,40.47,1,40.47,3,116.63,6,299.65,True,74.24.97.238,2025-09-06 17:00:41,2025-09-06 18:58:51,None,None,NaN,NaT,NaN
6,1352-3884-5289-2657,None,None,1,38.76,1,38.76,1,38.76,7,338.41,True,96.64.202.16,2025-09-06 18:58:51,2025-09-11 11:14:36,None,None,NaN,NaT,NaN
7,1352-3884-5289-2657,None,None,1,16.93,1,16.93,2,55.69,8,355.34,True,None,2025-09-11 11:14:36,2025-09-11 21:52:58,None,None,NaN,NaT,NaN
8,1352-3884-5289-2657,None,None,1,34.88,1,34.88,2,51.81,6,207.20,False,108.196.116.10,2025-09-11 21:52:58,2025-09-12 20:01:29,None,None,NaN,NaT,NaN
9,1352-3884-5289-2657,None,None,1,89.33,1,89.33,2,124.21,6,291.12,True,12.27.167.190,2025-09-12 20:01:29,2025-09-13 13:49:02,None,None,NaN,NaT,NaN


In [ ]:
selection = cc_trans_fg.select_except(['t_id', 'cc_num', 'merchant_id', 'account_id', 'ip_address', 'ts'])\
    .join(merchant_fg.select_features(), prefix="merchant_", on="merchant_id")\
    .join(subtree1, on="cc_num")

In [ ]:
df = selection.show(100)
df

In [ ]:
fv = fs.get_or_create_feature_view(name="cc_fraud_fv", 
                                   version=1, 
                                   description="features for a credit card fraud prediction model",
                                   query=selection,
                                   labels=['is_fraud']
                                  )

In [ ]:
test_start="2025-09-15 00:00"
X_train, X_test, y_train, y_test = fv.train_test_split(test_start=test_start)
X_train